In [1]:
%load_ext autoreload
%autoreload 2

from tokenizers import Tokenizer

import math
import random
import pandas as pd 
import numpy as np 
import pickle as pkl 
import plotly.graph_objects as go

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.utils import clip_grad_norm_

from sacrebleu.metrics import BLEU, CHRF, TER

from seq2seq import generate_data_loader, Seq2seqConfig, Seq2seq, init_weights, train, evaluate, plot_loss

## 1 - Hyperpatameters training

In [2]:
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    device = torch.device('cuda')
else : 
    device = torch.device('cpu')

tokenizer = Tokenizer.from_file("./data/preprocess/vocab.json")
config = Seq2seqConfig('./data/preprocess/vocab.json', device, hidden_size=1000, embedding_size=620, maxout_hidden_size = 500)

batch_size = 4
train_dataset, train_data_loader, dev_dataset, dev_data_loader, test_dataset, test_data_loader = generate_data_loader(batch_size, './data/preprocess/train.pkl', './data/preprocess/dev.pkl', './data/preprocess/test.pkl', tokenizer)

seq2seq = Seq2seq(config, device).to(device)
seq2seq.apply(init_weights)

loss_fn = nn.CrossEntropyLoss(ignore_index = config.padding_idx)
lr = 1e-05
optimizer = torch.optim.Adam(seq2seq.parameters(), lr = lr)

base_lr, max_lr, train_dataset_len = 0.0001, 0.001, train_dataset.__len__()
step_size = math.ceil(train_dataset_len / batch_size) * 2
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=base_lr, max_lr = max_lr, step_size_up=step_size, mode='triangular', cycle_momentum= False)

train_loss_set = []
dev_loss_set = []
dev_bleu_set = []
clip = 1.0
epochs = 8
beam_size = 3
for t in range(epochs):

    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(seq2seq, device, train_data_loader, optimizer, config, loss_fn, clip)
    dev_loss, dev_bleu = evaluate(seq2seq, device, tokenizer, dev_data_loader, config, loss_fn, beam_size, batch_size)
    train_loss_set.append(train_loss)
    dev_loss_set.append(dev_loss)
    dev_bleu_set.append(dev_bleu)

plot_loss(train_loss_set, dev_loss_set, dev_bleu_set)

Epoch 1
-------------------------------


KeyboardInterrupt: 